# Generate Input Data

In [3]:
import numpy as np
import pandas as pd
import random
#Generate 5 random numbers between 10 and 30
np.random.seed(0)
n_samples=1000
n_features = 5
df_XY=pd.DataFrame(data = np.random.normal(0,1, size=(n_samples, n_features)), columns = ['A','B','C','D','E'])
df_XY['Y']=list(np.random.randint(2, size=n_samples))
df_XY['YY']=list(np.random.randint(2, size=n_samples))
df_XY

##############################################################   
df_XY.shape
df_XY.head()
df_XY.to_csv('df_XY.csv',index=False)
##############################################################

In [4]:
df_XY

,A,B,C,D,E,Y,YY
0,1.764052,0.400157,0.978738,2.240893,1.867558,1,1
1,-0.977278,0.950088,-0.151357,-0.103219,0.410599,0,0
2,0.144044,1.454274,0.761038,0.121675,0.443863,0,0
3,0.333674,1.494079,-0.205158,0.313068,-0.854096,1,0
4,-2.552990,0.653619,0.864436,-0.742165,2.269755,0,1
...,...,...,...,...,...,...,...
995,1.711489,-1.820816,0.163495,-0.813117,-0.605355,0,0
996,-1.327524,-0.644172,1.908883,-0.563545,1.082473,1,0
997,-1.951911,2.441216,-0.017285,0.912282,1.239658,1,1
998,-0.573367,0.424889,-0.271260,-0.683568,-1.537438,1,1


# Download CI-VAE, other necessary packages and Load Packages

In [10]:
! rm -rf ci_vae
! rm bb.pt
! rm bb_residuals.pkl
! rm df_reconstructed.csv
! rm df_reconstructed_decoder.csv
! rm residuals.pdf
! rm results_dict.pkl
! rm df_latent.csv

rm: bb.pt: No such file or directory
rm: bb_residuals.pkl: No such file or directory
rm: df_reconstructed.csv: No such file or directory
rm: df_reconstructed_decoder.csv: No such file or directory
rm: residuals.pdf: No such file or directory
rm: results_dict.pkl: No such file or directory
rm: df_latent.csv: No such file or directory


In [11]:
! git clone https://github.com/monabiyan/ci_vae.git
! pip install umap-learn

Cloning into 'ci_vae'...
remote: Enumerating objects: 314, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 314 (delta 41), reused 50 (delta 22), pack-reused 245
Receiving objects: 100% (314/314), 46.61 MiB | 1.59 MiB/s, done.
Resolving deltas: 100% (197/197), done.


In [12]:
import pandas as pd
import numpy as np
import sklearn
import torch
import pickle

In [ ]:
import umap

In [13]:
import matplotlib.pyplot as plt
from ci_vae import ivae

# Set Necessary Parameters 

In [14]:
model_init=True
model_tobe_trained=True
save_address="bb"

kl_coef = 0.0001
reconst_coef = 1
classifier_coef = 0.1
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")



# Initialize the Model

In [15]:
obj1 = ivae.IVAE(df_XY = df_XY,
               latent_size = 2,
               reconst_coef = reconst_coef,
               kl_coef = kl_coef,
               classifier_coef = classifier_coef,
               test_ratio = 1)

if model_init:
    obj1.model_initialiaze()

## See The Model Architecture

In [16]:
print(obj1.model)

IVAE_ARCH(
  (encoder): Sequential(
    (0): Linear(in_features=5, out_features=20, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(20, eps=1e-05, momentum=0.2, affine=True, track_running_stats=True)
    (3): Dropout(p=0.05, inplace=False)
    (4): Linear(in_features=20, out_features=20, bias=True)
    (5): ReLU()
    (6): BatchNorm1d(20, eps=1e-05, momentum=0.2, affine=True, track_running_stats=True)
    (7): Dropout(p=0.05, inplace=False)
    (8): Linear(in_features=20, out_features=20, bias=True)
    (9): ReLU()
    (10): BatchNorm1d(20, eps=1e-05, momentum=0.2, affine=True, track_running_stats=True)
    (11): Dropout(p=0.05, inplace=False)
    (12): Linear(in_features=20, out_features=20, bias=True)
    (13): ReLU()
    (14): BatchNorm1d(20, eps=1e-05, momentum=0.2, affine=True, track_running_stats=True)
    (15): Dropout(p=0.05, inplace=False)
    (16): Linear(in_features=20, out_features=20, bias=True)
    (17): ReLU()
    (18): BatchNorm1d(20, eps=1e-05, momentum=0.2, affine=Tru

## See the Initialized Weights

In [17]:
for param in obj1.model.parameters():
    print(param)

Parameter containing:
tensor([[-0.1116, -0.0716,  0.1774, -0.0217,  0.1232],
        [-0.2089,  0.2904, -0.3151,  0.0018,  0.1702],
        [ 0.4039,  0.0059, -0.3560,  0.0748,  0.2586],
        [-0.0390, -0.4365, -0.1357,  0.1875,  0.0650],
        [ 0.0009, -0.2495, -0.1444,  0.3428, -0.4271],
        [-0.2311, -0.2492,  0.0734, -0.3642, -0.2027],
        [ 0.1185, -0.0714,  0.1192, -0.3464,  0.0147],
        [-0.4005,  0.0036, -0.4447,  0.2115, -0.2491],
        [-0.2210,  0.1760,  0.4244,  0.0452,  0.2285],
        [-0.2177,  0.1071,  0.1869, -0.2493,  0.0275],
        [ 0.3529,  0.3704,  0.3200, -0.1589, -0.3206],
        [ 0.2002, -0.1411, -0.2441,  0.3281,  0.3959],
        [ 0.3157,  0.1002, -0.1249,  0.2184, -0.4051],
        [-0.3974, -0.0537,  0.0541, -0.1319,  0.3722],
        [-0.1814, -0.2998,  0.1121, -0.4305,  0.1683],
        [-0.4133,  0.1166, -0.4253, -0.1438, -0.1429],
        [ 0.3261,  0.3583,  0.1989, -0.1723,  0.3408],
        [-0.0235,  0.2925, -0.0515, -0.1799

# RUN for Training

In [18]:
##########
if model_tobe_trained:
    lr=1e-2
    print(lr)
    obj1.model_training(epochs=100,learning_rate=lr)

    lr=1e-3
    print(lr)
    #obj.model_training(epochs=70,learning_rate=lr)

    lr=1e-3
    print(lr)
    #obj.model_training(epochs=200,learning_rate=lr)

    obj1.model_save(address=save_address+".pt")
    obj1.save_residuals(address=save_address+'_residuals.pkl')
    lr=1e-3
    print(lr)
    #obj.model_training(epochs=70,learning_rate=lr)

    lr=5e-4
    print(lr)
    obj1.model_training(epochs=50,learning_rate=lr)

    obj1.model_save(address=save_address+".pt")
    obj1.save_residuals(address=save_address+'_residuals.pkl')

    lr=1e-5
    print(lr)
    obj1.model_training(epochs=50,learning_rate=lr)

    lr=5e-6
    print(lr)
    obj1.model_training(epochs=50,learning_rate=lr)


0.01


/Users/mohsennabian/Desktop/ci_vae/example/ci_vae/ivae.py:372: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y=torch.tensor(torch.reshape(y, (-1,)), dtype=torch.long)
/Users/mohsennabian/Desktop/ci_vae/example/ci_vae/ivae.py:400: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(torch.reshape(y, (-1,)), dtype=torch.long)


====> Epoch: 1 total_train_loss: 1.027460 Total_test_loss: 0.866746 Total_BCE_test_loss: 0.789441 Total_KLD_test_loss: 0.000007 Total_CEP_test_loss: 0.077297
====> Epoch: 2 total_train_loss: 0.946405 Total_test_loss: 0.866250 Total_BCE_test_loss: 0.789049 Total_KLD_test_loss: 0.000009 Total_CEP_test_loss: 0.077192
====> Epoch: 3 total_train_loss: 0.914498 Total_test_loss: 0.865015 Total_BCE_test_loss: 0.789062 Total_KLD_test_loss: 0.000012 Total_CEP_test_loss: 0.075941
====> Epoch: 4 total_train_loss: 0.904479 Total_test_loss: 0.865420 Total_BCE_test_loss: 0.790327 Total_KLD_test_loss: 0.000018 Total_CEP_test_loss: 0.075076
====> Epoch: 5 total_train_loss: 0.895540 Total_test_loss: 0.866335 Total_BCE_test_loss: 0.791058 Total_KLD_test_loss: 0.000025 Total_CEP_test_loss: 0.075251
====> Epoch: 6 total_train_loss: 0.888908 Total_test_loss: 0.866321 Total_BCE_test_loss: 0.792112 Total_KLD_test_loss: 0.000033 Total_CEP_test_loss: 0.074176
====> Epoch: 7 total_train_loss: 0.882132 Total_test

====> Epoch: 56 total_train_loss: 0.706591 Total_test_loss: 0.682936 Total_BCE_test_loss: 0.612052 Total_KLD_test_loss: 0.001566 Total_CEP_test_loss: 0.069318
====> Epoch: 57 total_train_loss: 0.703663 Total_test_loss: 0.679614 Total_BCE_test_loss: 0.608684 Total_KLD_test_loss: 0.001614 Total_CEP_test_loss: 0.069316
====> Epoch: 58 total_train_loss: 0.705050 Total_test_loss: 0.675095 Total_BCE_test_loss: 0.604094 Total_KLD_test_loss: 0.001686 Total_CEP_test_loss: 0.069315
====> Epoch: 59 total_train_loss: 0.700464 Total_test_loss: 0.672194 Total_BCE_test_loss: 0.601132 Total_KLD_test_loss: 0.001753 Total_CEP_test_loss: 0.069309
====> Epoch: 60 total_train_loss: 0.698936 Total_test_loss: 0.671108 Total_BCE_test_loss: 0.600026 Total_KLD_test_loss: 0.001795 Total_CEP_test_loss: 0.069287
====> Epoch: 61 total_train_loss: 0.697718 Total_test_loss: 0.670977 Total_BCE_test_loss: 0.599886 Total_KLD_test_loss: 0.001822 Total_CEP_test_loss: 0.069269
====> Epoch: 62 total_train_loss: 0.695276 Tot

====> Epoch: 10 total_train_loss: 0.653674 Total_test_loss: 0.617592 Total_BCE_test_loss: 0.545988 Total_KLD_test_loss: 0.002329 Total_CEP_test_loss: 0.069276
====> Epoch: 11 total_train_loss: 0.659796 Total_test_loss: 0.616605 Total_BCE_test_loss: 0.544978 Total_KLD_test_loss: 0.002330 Total_CEP_test_loss: 0.069298
====> Epoch: 12 total_train_loss: 0.658288 Total_test_loss: 0.617324 Total_BCE_test_loss: 0.545741 Total_KLD_test_loss: 0.002306 Total_CEP_test_loss: 0.069276
====> Epoch: 13 total_train_loss: 0.654680 Total_test_loss: 0.616153 Total_BCE_test_loss: 0.544562 Total_KLD_test_loss: 0.002304 Total_CEP_test_loss: 0.069288
====> Epoch: 14 total_train_loss: 0.652106 Total_test_loss: 0.617154 Total_BCE_test_loss: 0.545583 Total_KLD_test_loss: 0.002306 Total_CEP_test_loss: 0.069265
====> Epoch: 15 total_train_loss: 0.653277 Total_test_loss: 0.616196 Total_BCE_test_loss: 0.544667 Total_KLD_test_loss: 0.002302 Total_CEP_test_loss: 0.069227
====> Epoch: 16 total_train_loss: 0.654791 Tot

====> Epoch: 15 total_train_loss: 0.648383 Total_test_loss: 0.607736 Total_BCE_test_loss: 0.536116 Total_KLD_test_loss: 0.002347 Total_CEP_test_loss: 0.069273
====> Epoch: 16 total_train_loss: 0.649860 Total_test_loss: 0.608563 Total_BCE_test_loss: 0.536931 Total_KLD_test_loss: 0.002350 Total_CEP_test_loss: 0.069282
====> Epoch: 17 total_train_loss: 0.650107 Total_test_loss: 0.608546 Total_BCE_test_loss: 0.536921 Total_KLD_test_loss: 0.002347 Total_CEP_test_loss: 0.069277
====> Epoch: 18 total_train_loss: 0.648802 Total_test_loss: 0.608038 Total_BCE_test_loss: 0.536404 Total_KLD_test_loss: 0.002366 Total_CEP_test_loss: 0.069268
====> Epoch: 19 total_train_loss: 0.649215 Total_test_loss: 0.607555 Total_BCE_test_loss: 0.535923 Total_KLD_test_loss: 0.002345 Total_CEP_test_loss: 0.069286
====> Epoch: 20 total_train_loss: 0.654112 Total_test_loss: 0.608220 Total_BCE_test_loss: 0.536573 Total_KLD_test_loss: 0.002363 Total_CEP_test_loss: 0.069284
====> Epoch: 21 total_train_loss: 0.650008 Tot

====> Epoch: 20 total_train_loss: 0.649577 Total_test_loss: 0.607696 Total_BCE_test_loss: 0.536051 Total_KLD_test_loss: 0.002365 Total_CEP_test_loss: 0.069280
====> Epoch: 21 total_train_loss: 0.650578 Total_test_loss: 0.606819 Total_BCE_test_loss: 0.535183 Total_KLD_test_loss: 0.002362 Total_CEP_test_loss: 0.069273
====> Epoch: 22 total_train_loss: 0.651342 Total_test_loss: 0.607002 Total_BCE_test_loss: 0.535328 Total_KLD_test_loss: 0.002393 Total_CEP_test_loss: 0.069281
====> Epoch: 23 total_train_loss: 0.646897 Total_test_loss: 0.607285 Total_BCE_test_loss: 0.535634 Total_KLD_test_loss: 0.002360 Total_CEP_test_loss: 0.069290
====> Epoch: 24 total_train_loss: 0.656645 Total_test_loss: 0.606954 Total_BCE_test_loss: 0.535276 Total_KLD_test_loss: 0.002402 Total_CEP_test_loss: 0.069276
====> Epoch: 25 total_train_loss: 0.651018 Total_test_loss: 0.607275 Total_BCE_test_loss: 0.535617 Total_KLD_test_loss: 0.002369 Total_CEP_test_loss: 0.069290
====> Epoch: 26 total_train_loss: 0.655692 Tot

# Save The Trained Model

In [ ]:
##########
print("running the neural network")
#run(obj1,save_address)
obj1.model_save(address=save_address+".pt")
obj1.save_residuals(address=save_address+'_residuals.pkl')

# Visualize Residuals

In [ ]:
obj1.plot_residuals(init_index=0)

# Load Model and Data

In [ ]:
from ci_vae import ivae
import pandas as pd
import numpy as np
import sklearn
import torch
import umap
import pickle
import matplotlib.pyplot as plt

In [ ]:
print("start of the code")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

##############################################################   
##############################################################
model_init=True
model_tobe_trained=False

model_init=True
model_file_address='./bb.pt'
save_address1="./"

df_XY=pd.read_csv('df_XY.csv')

## Load Saved Model

In [ ]:
obj1 = ivae.IVAE(df_XY = df_XY,
               reconst_coef = reconst_coef,
               latent_size = 10,
               kl_coef = kl_coef,
               classifier_coef = classifier_coef,
               test_ratio = 1)

obj1.model_initialiaze()

obj1.model_load(address="bb.pt")

## Print the Weights

In [ ]:
for param in obj1.model.parameters():
    print(param)

# Make Prediction of All Test Data

In [ ]:
with torch.no_grad():
    obj1.model.eval()

    #obj1.load_residuals(address='bb_residuals.pkl')
    print("model loaded")
    
    obj1.generate_test_results()
    print("test data generated")

# Comprehensive Checking of The Prediction Values vs. True Values. 

In [ ]:
print(obj1.x_last)

In [ ]:
print(obj1.x_pred)

In [ ]:
(np.abs(obj1.x_pred - obj1.x_last)).mean()

In [ ]:
(obj1.x_pred-obj1.x_last)

In [ ]:
print(obj1.y_last)

In [ ]:
print(obj1.y_pred)

In [ ]:
df_XY

In [ ]:
with torch.no_grad():
    obj1.model.eval()
    for x, y in obj1.testloader:
      x = x.to(device)
      print(x.size())
      print(x)
      # forward
      x_hat,y_hat, mu, logvar,z = obj1.model(x)
    
    df_reconstructed = pd.DataFrame(x_hat.cpu().detach().numpy(), columns=obj1.df_XY.drop(columns=['Y']).columns)
    print(df_reconstructed.shape)
    df_latent=pd.DataFrame(z.cpu().detach().numpy())
    
    obj1.model.eval()
    
    df_reconstructed_decoder=pd.DataFrame(obj1.model.decoder(z).cpu().detach().numpy(), columns=obj1.df_XY.drop(columns=['Y']).columns)

    df_reconstructed.to_csv('df_reconstructed.csv')
    df_latent.to_csv('df_latent.csv')
    df_reconstructed_decoder.to_csv('df_reconstructed_decoder.csv')
    print("Full_data_reconstructed...")
    
    print("========df_reconstructed========")
    print(df_reconstructed)
    print("========df_reconstructed_decoder========")
    print(df_reconstructed_decoder)
    print("========df_Original========")
    print(df_XY)
    
    
    


# Checking Linear Separability of Data on Lower Dimensioanl Space

In [ ]:
print("regression analysis")
obj1.regression_analysis(obj1.zs,df_XY['Y'])

# Visualize Data on Lower Dimensional Space

In [ ]:
print("calculate tsne_umap_pca")
tsne_mat,umap_mat,pca_mat,Y=obj1.calculate_lower_dimensions(obj1.zs,obj1.y_last,N=100)
obj1.plot_lower_dimension(tsne_mat,Y,projection='3d',save_str='tsne3d.pdf')
obj1.plot_lower_dimension(tsne_mat,Y,projection='2d',save_str='tsne2d.pdf')
obj1.plot_lower_dimension(umap_mat,Y,projection='3d',save_str='umap3d.pdf')
obj1.plot_lower_dimension(umap_mat,Y,projection='2d',save_str='umap2d.pdf')
obj1.plot_lower_dimension(pca_mat,Y,projection='3d',save_str='pca3d.pdf')
obj1.plot_lower_dimension(pca_mat,Y,projection='2d',save_str='pca2d.pdf')

print("finished")


# Perform Interpolation across all groups (Y) and all features from YY=0 to YY=1

In [ ]:
ff = obj1.traversal_all_groups(traversal_step=5)

# See the interpolation results

In [ ]:
with open('results_dict.pkl', 'rb') as f:
    ff = pickle.load(f)

In [ ]:
ff['med']['0']

In [ ]:
ff['mean']['1']

In [ ]:
plt.plot(ff['med']['1']['B'])

In [ ]:
plt.plot(ff['mean']['1']['B'])

In [ ]:
plt.plot(ff['med']['0']['C'])

In [ ]:
import matplotlib.pyplot as plt
plt.plot(ff['mean']['0']['C'])

# Generate Synthetic Data for a Given Class

In [ ]:
bb = obj1.synthetic_single_group(group_id=0,nr_of_synthetic=1000)

In [ ]:
bb.shape

In [ ]:
bb